In [1]:
pip install -U tensorflow

^C
Note: you may need to restart the kernel to use updated packages.


  Using cached tensorflow-2.11.0-cp39-cp39-win_amd64.whl (1.9 kB)
  Using cached tensorflow_intel-2.11.0-cp39-cp39-win_amd64.whl (266.3 MB)


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\sandr\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python39\\site-packages\\tensorflow\\include\\external\\com_github_grpc_grpc\\src\\core\\ext\\filters\\client_channel\\lb_policy\\grpclb\\client_load_reporting_filter.h'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths


[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: C:\Users\sandr\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
pip install 'scipy>=1.8'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install 'networkx<2.7'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
 
drive.mount('/content/drive') 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
# import tensorflow as tf

initial_G = nx.read_edgelist("/content/drive/MyDrive/ART_Inv/GNN/Biogrid/biogrid_minimum.edgelist")  
G = nx.Graph(initial_G)
A = nx.adjacency_matrix(G).todense()

print("Adjacency Matrix:")
print(A)

# diagonal matrix
D = np.diag(A.sum(axis=1))

# graph laplacian
L = D-A

# eigenvalues and eigenvectors
vals, vecs = np.linalg.eig(L)

# create data frame with the second and the third minimum eigenvalue
df = pd.DataFrame(vecs.T, 
                   columns=vals)
print(df)
print(vals)
path ='/content/drive/MyDrive/ART_Inv/GNN/Biogrid/eigenvalues.csv'
df = pd.read_csv(path)
first_eigen_val = df.columns[np.where(df.columns == np.amin(df.columns))[0]][0]
df = df.drop([first_eigen_val], axis=1)

second_eigen_val = df.columns[np.where(df.columns == np.amin(df.columns))[0]][0]
df2 = pd.DataFrame(df[second_eigen_val], columns=[second_eigen_val])
df = df.drop([second_eigen_val], axis=1)

third_eigen_val = df.columns[np.where(df.columns == np.amin(df.columns))[0]][0]
col = df[third_eigen_val]

df2[third_eigen_val] =  col

print("#################################################")
print(df2)
print(np.abs(np.round(df2[second_eigen_val])).max()+1)
print(np.abs(np.round(df2[third_eigen_val])).max()+1)

for i in range(len(df2)):
  plt.plot(df2[third_eigen_val][i], df2[second_eigen_val][i], marker="x",  markersize=7, color="green")

# plt.show()

print("#################################################")

print(df2[third_eigen_val].min())
df2[third_eigen_val] = df2[third_eigen_val]+abs(df2[third_eigen_val].min())
df2[second_eigen_val] = df2[second_eigen_val]+abs(df2[second_eigen_val].min())
print(df2)
for i in range(len(df2)):
  plt.plot(df2[third_eigen_val][i], df2[second_eigen_val][i], marker="x",  markersize=7, color="green")
# plt.show()

df2 = df2*100
df2 = df2.astype("int32")
print(df2)
matrix = np.zeros((df2[third_eigen_val].max()+1, df2[second_eigen_val].max()+1))
print(matrix.size)
# df1 = df2[df2.duplicated(keep=False)]

# print (df1)
heat = df2.groupby(df2.columns.tolist()).size().reset_index().rename(columns={0:'records'})
print(heat)

matrix[heat[third_eigen_val], heat[second_eigen_val]] = heat["records"]
print(matrix)
import seaborn as sns
# sns.heatmap(matrix)
# plt.show()

######################## MAPEO DE GENES ########################


matrix = np.zeros((100, 100))
path ='/content/drive/MyDrive/ART_Inv/GNN/Biogrid/clinic_and_RNA_data_raw.csv'
data = pd.read_csv(path)
# print(data)
X = []
Y = []
for patient in range(len(data)): # Recorro pacientes
  # print(patient)
  for i in range(len(df2)):
    # print(df2.iloc[i])
    # print(df2[third_eigen_val][i])
    # print(df2[second_eigen_val][i])
    # print(list(G.nodes)[i])
    # print(data[list(G.nodes)[i]][patient])
    matrix[df2[third_eigen_val][i], df2[second_eigen_val][i]] = data[list(G.nodes)[i]][patient]
    matrix = matrix.astype("int32")
  X.append(matrix)
  # print(data.PFS[patient])2
  if data.PFS[patient] < 3:
    Y.append(0)
  else:
    Y.append(1)
print(X[0])

#################################### CNN #####################################


import tensorflow as tf
from tensorflow.keras import activations

from tensorflow.keras import datasets, layers, models
# input = tf.keras.layers.Input(shape=(None, None, 100, 100))

model = models.Sequential()
model.add(layers.Conv2D(512, (5, 5), activation='relu', input_shape = (100,100,1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(256))
model.add(layers.Dense(256))
model.add(layers.Dense(128))
model.add(layers.Dense(1, activation='hard_sigmoid'))

print(model.summary())
# print(model.output_shape)


from sklearn.model_selection import train_test_split 
train_data, test_data, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, stratify=Y)
print("train data")
print(len(train_data))
from sklearn.model_selection import StratifiedKFold # import KFold
kf=StratifiedKFold(n_splits=5, random_state=None, shuffle=False)

val_avg = []
test_avg = []
test_f1_score = []
for train_index, val_index in kf.split(train_data, Y_train):
    train_dataset=[]
    val_dataset=[]
    train_y_dataset = []
    val_y_dataset = []
    print("TRAIN: ", train_index, "TEST:", val_index)
    for i in train_index:
        train_dataset.append(train_data[i])
        train_y_dataset.append(Y_train[i])
    for i in val_index:
        val_dataset.append(train_data[i])
        val_y_dataset.append(Y_train[i])

    # print(len(train_dataset))
    # print(len(val_dataset))
    train_y_dataset=np.array(train_y_dataset)
    val_y_dataset=np.array(val_y_dataset)
    train_dataset=np.array(train_dataset)
    val_dataset=np.array(val_dataset)
    # train_dataset=train_dataset.reshape(251,100,100)

    # print("len")
    # print(len(train_y_dataset))
    # print(len(train_dataset))

    model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate= 0.00001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    history = model.fit(train_dataset, train_y_dataset, epochs=30, 
                    validation_data=(val_dataset, val_y_dataset), batch_size = 8)

Adjacency Matrix:
[[0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (8,11,12,21,30,50) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 96, 96, 512)       13312     
                                                                 
 max_pooling2d_24 (MaxPoolin  (None, 48, 48, 512)      0         
 g2D)                                                            
                                                                 
 conv2d_25 (Conv2D)          (None, 46, 46, 256)       1179904   
                                                                 
 max_pooling2d_25 (MaxPoolin  (None, 23, 23, 256)      0         
 g2D)                                                            
                                                                 
 conv2d_26 (Conv2D)          (None, 21, 21, 128)       295040    


In [ ]:
test_data=np.array(test_data)

In [ ]:
test_data = test_data.tolist()

In [ ]:
model.evaluate(test_data, Y_test)